In [1]:
### Package Setups
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
from tensorflow import keras
import time

from functions_dataCreation import *

from tensorflow.keras.callbacks import Callback

import wandb
from wandb.keras import WandbCallback

### Data Loading
train = createIODataset(50,'../data/Train')
test = createIODataset(4,'../data/Test')

train = train.repeat(-1)
train = train.shuffle(buffer_size=10240,reshuffle_each_iteration=True)
train = train.batch(256,drop_remainder=True)
train = train.prefetch(4)

test = test.repeat(-1)
test = test.shuffle(buffer_size=10240,reshuffle_each_iteration=True)
test = test.batch(256,drop_remainder=True)
test = test.prefetch(1)

1 Physical GPUs, 1 Logical GPUs


In [2]:
#MobileNetV2 expects pixel vaues in [-1,1], but at this point, the pixel values in your images are in [0-255]. 

preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input


In [3]:
# Create the base model from the pre-trained model MobileNet V2
IMG_SIZE = 128
base_model = tf.keras.applications.MobileNetV2(input_shape=(IMG_SIZE,IMG_SIZE,3),include_top=False,weights='imagenet')

In [4]:
# Freezing (by setting layer.trainable = False) prevents the weights in a given layer from being updated during training.
base_model.trainable = False
base_model.summary()


Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

In [5]:
#This feature extractor converts each 128x128x3 image into a 5x5x1280 block of features. Let's see what it does to an example batch of images. (Batch size being 256)

image_batch, label_batch = next(iter(train))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

(256, 4, 4, 1280)


In [6]:
# Add a classification head
# To generate predictions from the block of features, average over the spatial 5x5 spatial locations, using a tf.keras.layers.GlobalAveragePooling2D layer to convert the features to a single 1280-element vector per image.

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

(256, 1280)


In [7]:
# Apply a tf.keras.layers.Dense layer to convert these features into a single prediction per image. You don't need an activation function here because this prediction will be treated as a logit, or a raw prediction value. Positive numbers predict class 1, negative numbers predict class 0.

# prediction_layer = tf.keras.layers.Dense(1)
prediction_layer = tf.keras.layers.Dense(1,activation='sigmoid')
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

(256, 1)


In [8]:
# Build a model by chaining together the rescaling, base_model and feature extractor layers using the Keras Functional API. As previously mentioned, use training=False as our model contains a BatchNormalization layer.

inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = preprocess_input(inputs)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.4)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [9]:
# Compile the model before training it. Since there are two classes, use a binary cross-entropy loss with from_logits=True since the model provides a linear output.

METRICS = [keras.metrics.Precision(name='precision'),keras.metrics.Recall(name='recall'),keras.metrics.AUC(name='auc'),]

base_learning_rate = 1e-4
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
            #   loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              loss=tf.keras.losses.binary_crossentropy,
              metrics=METRICS)

model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
tf_op_layer_RealDiv (TensorF [(None, 128, 128, 3)]     0         
_________________________________________________________________
tf_op_layer_Sub (TensorFlowO [(None, 128, 128, 3)]     0         
_________________________________________________________________
mobilenetv2_1.00_128 (Functi (None, 4, 4, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                

In [10]:
# The 2.5M parameters in MobileNet are frozen, but there are 1.2K trainable parameters in the Dense layer. These are divided between two tf.Variable objects, the weights and biases.

len(model.trainable_variables)


2

In [11]:
print("weights:", len(model.weights))
print("trainable_weights:", len(model.trainable_weights))
print("non_trainable_weights:", len(model.non_trainable_weights))


weights: 262
trainable_weights: 2
non_trainable_weights: 260


In [12]:
run = wandb.init(project="candlestick-CNN", name = 'TL50FilesShuffle10KSteps400_AdamLR1e-4_Base' ,reinit= True)

history = model.fit(train
                ,epochs=100
                ,steps_per_epoch=10240*10/256 #800
                ,verbose=1
                ,validation_data=test                
                ,validation_freq = 5
                ,validation_steps = 10
                ,callbacks=[WandbCallback()]
                )


run.finish()


Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: amitagni (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.10 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Epoch 1/100
400/400 [==============================] - 31s 77ms/step - loss: 0.6843 - precision: 0.2268 - recall: 0.1435 - auc: 0.5002
Epoch 2/100
400/400 [==============================] - 31s 78ms/step - loss: 0.6434 - precision: 0.2569 - recall: 0.1025 - auc: 0.5084
Epoch 3/100
400/400 [==============================] - 31s 78ms/step - loss: 0.5998 - precision: 0.2363 - recall: 0.0573 - auc: 0.5091
Epoch 4/100
400/400 [==============================] - 31s 78ms/step - loss: 0.5939 - precision: 0.2713 - recall: 0.0448 - auc: 0.5140
Epoch 5/100
400/400 [==============================] - 34s 85ms/step - loss: 0.5663 - precision: 0.2471 - recall: 0.0172 - auc: 0.5189 - val_loss: 0.5754 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5085
Epoch 6/100
400/400 [==============================] - 31s 78ms/step - loss: 0.5693 - precision: 0.3092 - recall: 0.0136 - auc: 0.5248
Epoch 7/100
400/400 [==============================] - 31s 78ms/step - loss: 0.5552 - precision: 0.2

epoch,99
loss,0.54686
precision,1.0
recall,8e-05
auc,0.55366
_step,99
_runtime,3165
_timestamp,1605122025
val_loss,0.54299
val_precision,0.0
val_recall,0.0


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▄▃▂▂▂▂▂▁▁▂▂▁▁▁▂▂▁▁▂▂▁▁▂▂▁▂▂▁▁▂▂▁▁▁▁▁▁▁▂
precision,▃▃▃▃▃▃▁▅▁██▁█▁▁▁▁▆▁█▁█▆█▁▆▃██▆▄▅▅▅▇▆▆█▆█
recall,█▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
auc,▁▂▄▄▅▆▆▆▇▇▆▆▇▇▇▇▇▇█▇▇██▇▇▇▇▇▇▇▇▇██▇▇██▇▇
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_loss,█▂▄▄█▂▂▄▅▅▂▇▂▆▇▅▃▆▆▁
val_precision,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_recall,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


Epoch 10/10
800/800 [==============================] - 94s 117ms/step - loss: 0.5294 - precision: 0.6667 - recall: 5.6124e-04 - auc: 0.5652 - val_loss: 0.6202 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5234


### Fine tuning 

In [13]:
# unfreeze the base_model and set the bottom layers to be un-trainable

base_model.trainable = True

# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False
  

Number of layers in the base model:  155


In [14]:
# As you are training a much larger model and want to readapt the pretrained weights, it is important to use a lower learning rate at this stage. Otherwise, your model could overfit very quickly.

# model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
#               optimizer = tf.keras.optimizers.RMSprop(lr=base_learning_rate/10),
#               metrics=['accuracy'])

model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate/100),
            #   loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              loss=tf.keras.losses.binary_crossentropy,
              metrics=METRICS)

model.summary()



Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
tf_op_layer_RealDiv (TensorF [(None, 128, 128, 3)]     0         
_________________________________________________________________
tf_op_layer_Sub (TensorFlowO [(None, 128, 128, 3)]     0         
_________________________________________________________________
mobilenetv2_1.00_128 (Functi (None, 4, 4, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                

In [15]:
# Continue training the model
# If you trained to convergence earlier, this step will improve your accuracy by a few percentage points.

# fine_tune_epochs = 10
# total_epochs =  initial_epochs + fine_tune_epochs

# history_fine = model.fit(train_dataset,
#                          epochs=total_epochs,
#                          initial_epoch=history.epoch[-1],
#                          validation_data=validation_dataset)

run = wandb.init(project="candlestick-CNN", name = 'TL50FilesShuffle10KSteps400_AdamLR1e-6_Finetuned' ,reinit= True)

history = model.fit(train
                ,epochs=800
                ,steps_per_epoch=10240*10/256 #800
                ,initial_epoch=history.epoch[-1]
                ,verbose=1
                ,validation_data=test                
                ,validation_freq = 5
                ,validation_steps = 5
                ,callbacks=[WandbCallback()]
                )


run.finish()

                         

wandb: wandb version 0.10.10 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Epoch 2/1000
400/400 [==============================] - 45s 112ms/step - loss: 0.5639 - precision: 0.2520 - recall: 0.0333 - auc: 0.5110
Epoch 3/1000
400/400 [==============================] - 45s 113ms/step - loss: 0.5536 - precision: 0.3125 - recall: 6.1828e-04 - auc: 0.5221
Epoch 4/1000
400/400 [==============================] - 45s 113ms/step - loss: 0.5528 - precision: 0.3333 - recall: 8.2001e-05 - auc: 0.5223
Epoch 5/1000
400/400 [==============================] - 48s 120ms/step - loss: 0.5492 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5297 - val_loss: 0.5468 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5443
Epoch 6/1000
400/400 [==============================] - 45s 113ms/step - loss: 0.5522 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5303
Epoch 7/1000
400/400 [==============================] - 45s 113ms/step - loss: 0.5476 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5328
Epoch 8/1000
400/400 [==============================] - 

KeyboardInterrupt: 

Epoch 20/20
800/800 [==============================] - 116s 145ms/step - loss: 0.2530 - precision: 0.8447 - recall: 0.6638 - auc: 0.9469 - val_loss: 0.9897 - val_precision: 0.3161 - val_recall: 0.1632 - val_auc: 0.5052